In [1]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
combined_data = pd.read_csv('../data/intermediates/pre_acpca1.csv')
combined_data.set_index('FIPS', inplace=True)
print(combined_data.shape)
combined_data.head()

(211, 21)


,baseline,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Mar Temp AVG / F,Apr Temp AVG / F,May Temp AVG / F,...,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,HospCt,Deaths,normalized_deaths
FIPS,,,,,,,,,,,,,,,,,,,,,
1055.0,0.411541,0.116127,0.25,0.361022,0.123984,0.135802,0.321642,0.697723,0.721448,0.778049,...,0.203494,0.325792,0.294336,0.358494,0.567540,0.362510,0.075155,0.258374,0.020576,0.119644
1081.0,0.260364,0.138024,0.25,0.169329,0.457317,0.104938,0.390932,0.757764,0.743733,0.802439,...,0.056305,0.343891,0.586319,0.144048,0.571650,0.451833,0.609583,0.046170,0.082305,0.193068
1095.0,0.483091,0.097183,0.50,0.476038,0.154472,0.092593,0.344739,0.672878,0.696379,0.768293,...,0.162744,0.334842,0.241579,0.364819,0.680389,0.659251,0.086210,0.278906,0.012346,0.100503
1097.0,0.419447,0.202669,0.00,0.306709,0.229675,0.172840,0.299401,0.846791,0.813370,0.865854,...,0.145504,0.303167,0.505194,0.328976,0.607318,0.520245,0.119985,0.102618,0.144033,0.118461
1101.0,0.281413,0.175975,0.00,0.325879,0.416667,0.129630,0.402908,0.792961,0.782730,0.848780,...,0.123487,0.271493,0.275549,0.385057,0.597948,0.570767,0.176654,0.159904,0.012346,0.036609


## Define data matrix and confounding matrix

In [3]:
# confounders = ['baseline', 'Apr Temp AVG / F', 'Total households!!Average household size', 'Jun Temp AVG / F', 'HospCt', \
#                'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)', \
#                'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)', \
#               'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten', 'Med_HH_Income_Percent_of_State_Total_2018',\
#               'Density per square mile of land area - Population', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool']
confounders = ['baseline']
X = combined_data.drop(columns=['Deaths', 'normalized_deaths'], inplace=False)
X = X.drop(columns=confounders, inplace=False).to_numpy() # 'Density per square mile of land area - Population',
X_rows, X_cols = X.shape
Y = combined_data.filter(confounders).to_numpy()
# , 'Density per square mile of land area - Population'
# Y = Y.reshape(len(Y), 1)

In [4]:
density = combined_data['Density per square mile of land area - Population']
density.mean()

0.19912413516874522

In [5]:
0.115692 - 0.19967853479853478

-0.08398653479853478

## Center X and Y

In [6]:
X_mean = X.T.mean(axis=1)
X = X - X_mean
Y_mean = Y.T.mean(axis=1)
Y = Y - Y_mean

## Covariance/ objective funciton matrix

In [7]:
lam = 20 # lambda
K = Y @ Y.T
cov = X.T @ X - lam * X.T @ K @ X
eigenvals, eigenvecs = eig(cov.T)
print(eigenvecs.shape)
P = eigenvecs.T.dot(X.T)
P = P.T

(18, 18)


In [8]:
print(eigenvals.shape)
eigenvals[2]

(18,)


17.76047584274622

In [9]:
print(eigenvecs.shape)
print(eigenvecs[:5,:])
eigenvecs[:5,2]

(18, 18)
[[ 0.15996185 -0.11193627  0.13407681  0.35588711 -0.33141919  0.67091248
  -0.14326524  0.37521146  0.13053567 -0.10444245  0.04856352  0.18338131
  -0.0244023  -0.17233049 -0.00902546  0.02669411 -0.03079859 -0.00294754]
 [ 0.08940355  0.03967093 -0.32284993 -0.53176341  0.45732349  0.23807986
  -0.357487    0.38004761  0.01160417  0.10417748  0.18556286  0.10085887
  -0.06044376 -0.00233744 -0.01785477 -0.01988806 -0.03986511  0.06174733]
 [-0.04165096  0.17649566 -0.44542805  0.18001371 -0.19642887 -0.13993644
   0.11141904  0.09261607  0.42870734  0.28454098  0.00213256  0.20661735
  -0.38456999  0.15483266  0.0295775  -0.10274614  0.36816095  0.17565535]
 [ 0.32952995 -0.2715786   0.30490306  0.2124642   0.24395836 -0.1287343
  -0.15554268  0.28807828  0.02256912  0.32374828 -0.39070799 -0.19810948
  -0.01139583  0.42182253 -0.01647438 -0.07288246  0.06712992  0.10440845]
 [ 0.14062081  0.02010026 -0.25497752 -0.0077073  -0.16390992 -0.03575002
  -0.08310196  0.01022266 

array([ 0.13407681, -0.32284993, -0.44542805,  0.30490306, -0.25497752])

In [10]:
print(P.shape)
print(P[:5,2])

(211, 18)
[-0.33009537 -0.18799058 -0.50754352 -0.30969219 -0.19126676]


In [11]:
column_names = ['PC' + str(i) for i in range(1, X_cols+1)]
column_names

['PC1',
 'PC2',
 'PC3',
 'PC4',
 'PC5',
 'PC6',
 'PC7',
 'PC8',
 'PC9',
 'PC10',
 'PC11',
 'PC12',
 'PC13',
 'PC14',
 'PC15',
 'PC16',
 'PC17',
 'PC18']

In [12]:
projected = pd.DataFrame(data = P, columns=column_names)
PCs = pd.DataFrame(data=eigenvecs, columns=column_names)
eigenvalues = pd.DataFrame(data=eigenvals.reshape(1, X_cols), columns=column_names)
    

In [13]:
projected.to_csv('../data/intermediates/post_acpca_ind.csv', index=False)
projected.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18
0,-0.344782,-0.411385,-0.330095,-0.210771,-0.091091,0.021912,0.114324,-0.003376,0.036926,-0.019196,0.004712,0.093879,0.001751,0.036212,0.012772,-0.032177,-0.035903,-0.004168
1,0.072745,-0.727607,-0.187991,0.088592,0.199029,-0.204161,-0.163453,-0.060373,-0.021687,-0.070992,0.157642,0.066372,-0.066797,-0.076628,-0.024835,0.028554,-0.046145,-0.004432
2,-0.409617,-0.291495,-0.507544,-0.204880,0.072448,0.010730,-0.023235,0.214548,0.036939,-0.073444,0.012302,-0.015431,-0.151201,0.054053,0.004583,0.018669,0.004643,0.016965
3,-0.370313,-0.656632,-0.309692,0.121011,-0.162317,-0.003072,0.004115,-0.131009,-0.121696,-0.001256,0.015284,0.009097,-0.021708,0.002437,-0.032741,-0.016255,0.029211,-0.007044
4,-0.344726,-0.643939,-0.191267,0.121921,-0.065601,-0.088160,0.131033,0.081170,-0.054247,-0.040385,-0.061953,-0.075373,-0.056077,0.067255,-0.021235,-0.010428,0.064982,0.035943


In [14]:
PCs.to_csv('../data/intermediates/PCs_ind.csv', index=False)
PCs.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18
0,0.159962,-0.111936,0.134077,0.355887,-0.331419,0.670912,-0.143265,0.375211,0.130536,-0.104442,0.048564,0.183381,-0.024402,-0.172330,-0.009025,0.026694,-0.030799,-0.002948
1,0.089404,0.039671,-0.322850,-0.531763,0.457323,0.238080,-0.357487,0.380048,0.011604,0.104177,0.185563,0.100859,-0.060444,-0.002337,-0.017855,-0.019888,-0.039865,0.061747
2,-0.041651,0.176496,-0.445428,0.180014,-0.196429,-0.139936,0.111419,0.092616,0.428707,0.284541,0.002133,0.206617,-0.384570,0.154833,0.029577,-0.102746,0.368161,0.175655
3,0.329530,-0.271579,0.304903,0.212464,0.243958,-0.128734,-0.155543,0.288078,0.022569,0.323748,-0.390708,-0.198109,-0.011396,0.421823,-0.016474,-0.072882,0.067130,0.104408
4,0.140621,0.020100,-0.254978,-0.007707,-0.163910,-0.035750,-0.083102,0.010223,-0.010620,0.187527,-0.140945,-0.223018,0.630369,-0.359661,0.027905,0.048181,0.227828,0.439300


In [15]:
eigenvalues.to_csv('../data/intermediates/eigenvalues_ind.csv', index=False)
eigenvalues.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18
0,-339.209686,30.923555,17.760476,12.507342,7.687934,4.448063,3.824673,3.458216,2.323821,2.092037,1.638841,1.524806,1.145982,1.030967,0.114114,0.29321,0.494998,0.462995
